In [1]:
from __future__ import print_function 

%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf
from tensorflow.keras import backend as K
K.clear_session()

import pandas as pd
import pickle
import gc, re, copy
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

# Project imports 
from data.tasks import *
from data.label_shift import label_shift_linear, plot_labeldist, plot_splitbars
from experiments.models import *
from experiments.training import *
from bounds.bounds import *
from util.batch import *
from util.kl import *
from util.misc import *


project_folder=os.getcwd()+"/" ## change if necessary

# Load data & Run evaluation 

In [1]:
task = 2
architecture="lenet"
delta=0.05
binary=True
image_size=32
batch_size=32
alphas=[0]#0,0.3]
sigmas=[[3,3]]
seed=1
for alpha in alphas:
    print("alpha:"+str(alpha))
    if alpha==0:
        if task==7:
            source_generator, target_generator=load_task(task=task,alpha=alpha,architecture=architecture,binary=True,image_size=image_size,seed=seed)
        else:
            x_source,y_source,x_target,y_target,iw_source, iw_target=load_task(task)
            x_bound=x_source
            y_bound=y_source
    else:
        if task==7:
            prior_generator, bound_generator, target_generator=load_task(task=task,alpha=alpha,architecture=architecture,binary=True,image_size=image_size,seed=seed) ## create bound/prior split here as own generators
        else:
            x_source,y_source,x_target,y_target, iw_source, iw_target=load_task(task)
            x_bound, x_prior, y_bound , y_prior = train_test_split(x_source,y_source,test_size=alpha,random_state=seed)
            iw_bound, iw_prior, _,_ = train_test_split(iw_source,y_source,test_size=alpha,random_state=seed)
            print(iw_target)
    for sigma in sigmas:    
        print("sigma:"+str(sigma))
        paths=posterior_checkpoints(task, alpha, binary=binary,architecture=architecture,seed=seed,image_size=image_size)
        for p in paths:
            if alpha==0:
                if task==7:
                    results=compute_bound_parts(task, p, bound_generator=bound_generator, target_generator=target_generator, alpha=alpha, delta=0.05, bound='germain', binary=True, n_classifiers=5, sigma=sigma, seed=seed,batch_size=batch_size,architecture=architecture,image_size=image_size)
                else:
                    results=compute_bound_parts(task,p,x_bound=x_bound,y_bound=y_bound,x_target=x_target,y_target=y_target,alpha=alpha,sigma=sigma,seed=seed,binary=binary,architecture=architecture,image_size=image_size,batch_size=batch_size, iw_bound=iw_source, iw_target=iw_target)
                checkpoint = results['checkpoint'].values.ravel()[0]

                if binary:
                    result_path=project_folder+"results/"+"task"+str(task)+"/Binary/"+str(architecture)+"/"+str(image_size)+"_"+str(int(100*alpha))+"_"+str(sigma[0])+str(sigma[1])+'_'+str(seed)+'_'+checkpoint+'_results.pkl'
                else:
                    result_path=project_folder+"results/"+"task"+str(task)+"/"+str(architecture)+"/"+str(image_size)+"_"+str(int(100*alpha))+\
                    "_"+str(sigma[0])+str(sigma[1])+'_'+str(seed)+'_'+checkpoint+'_results.pkl'
            else:
                if task==7:
                    results=compute_bound_parts(task, p, bound_generator=bound_generator, target_generator=target_generator, alpha=alpha, delta=0.05, bound='germain', binary=True, n_classifiers=5, sigma=sigma, seed=seed,batch_size=batch_size,architecture=architecture,image_size=image_size)
                else:
                    results=compute_bound_parts(task,p,x_bound=x_bound,y_bound=y_bound,x_target=x_target,y_target=y_target,alpha=alpha,sigma=sigma,seed=seed,binary=binary,architecture=architecture,image_size=image_size,batch_size=batch_size, iw_bound=iw_bound, iw_target=iw_target)
                checkpoint = results['checkpoint'].values.ravel()[0]

                if binary:
                    result_path=project_folder+"results/"+"task"+str(task)+"/Binary/"+str(architecture)+"/"+str(image_size)+"_"+str(int(100*alpha))+"_"+str(sigma[0])+str(sigma[1])+'_'+str(seed)+'_'+checkpoint+'_results.pkl'
                else:
                    result_path=project_folder+"results/"+"task"+str(task)+"/"+str(architecture)+"/"+str(image_size)+"_"+str(int(100*alpha))+\
                    "_"+str(sigma[0])+str(sigma[1])+'_'+str(seed)+'_'+checkpoint+'_results.pkl'

            # Create dir
            os.makedirs(os.path.dirname(result_path), exist_ok=True)
            results.to_pickle(result_path)

alpha:0
mean, variance 0.36348352 70.18035
---------------Load MNIST----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)


mean, variance 1.1809415 74.36859
---------------Load MNIST-M----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)
[array([6328., 6565., 5243., 4761., 3981., 3157., 2866., 2431., 1707.,
       1160.], dtype=float32), array([ 576., 1313., 1748., 2381., 2844., 3157., 4012., 4862., 5119.,
       5799.], dtype=float32), array([ 575., 1312., 1747., 2380., 2843., 3156., 4010., 4862., 5118.,
       5798.], dtype=float32), array([6327., 6564., 5242., 4760., 3980., 3156., 2864., 2431., 1706.,
       1159.], dtype=float32)]
[[0.09102402, 0.2, 0.3333969, 0.500105, 0.7143934, 1.0, 1.3998604, 2.0, 2.9988284, 4.999138], [11.003478, 5.003049, 3.0005724, 2.0, 1.3999296, 1.0, 0.71421444, 0.5, 0.33333334, 0.19989651]]
[0.09102402 0.7143934  0.2        ... 0.71421444 0.33333334 0.19989651]


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/adam/anaconda3/envs/keras/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-a853e362e76f>", line 35, in <module>
    results=compute_bound_parts(task,p,x_bound=x_bound,y_bound=y_bound,x_target=x_target,y_target=y_target,alpha=alpha,sigma=sigma,seed=seed,binary=binary,architecture=architecture,image_size=image_size,batch_size=batch_size, iw_bound=iw_source, iw_target=iw_target)
  File "/home/adam/Code/pacbayes_bounds/bounds/bounds.py", line 176, in compute_bound_parts
    e_s, e_t, d_sx, d_tx, e_s_std, e_t_std, d_sx_std, d_tx_std, train_error, target_error, error_std, target_error_std,iw_error,target_iw_error, iw_std=draw_classifier_and_calculate_errors(w_s,sigma,n_classifiers,posterior_model,x_bound,y_bound,x_target,y_target,iw_bound=iw_bound, iw_target=iw_target)
  File "/home/adam/Code/pacbayes_bounds/bounds/bounds.py", li

TypeError: object of type 'NoneType' has no len()